In [ ]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import tqdm
import networkx as nx

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.corpora import Dictionary
from gensim.test.utils import datapath
from os import listdir, mkdir, remove
from os.path import isfile, join
from numpy import genfromtxt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


## Используемые и полученные данные:


*   Стартовый лемматизированный корпус: https://drive.google.com/file/d/1-C5vzhVDrJYgRdUGwNtj9wUTfooEVFaa/view?usp=sharing 
*  (Источник: https://www.gazeta.ru)

*   База данных городов:  https://drive.google.com/file/d/1VI6PqUXXNYD_JH_kncaKU_JOYpm2OjrK/view?usp=sharing 
*   (Источник: https://on55.ru/articles/2) 
*   Полученные таблицы координат: https://drive.google.com/drive/folders/1174tmqmjhcO-YRJrbT0YZC94msY1n8vy?usp=sharing


## Создаем модель на основе лемматизированного корпуса

In [ ]:
path = "/content/gdrive/MyDrive/lem_corpora/gazeta_ru-2018.csv"
data = pd.read_csv(path)

cities = pd.read_csv('/content/gdrive/MyDrive/koord_russia.csv', encoding='cp1251', delimiter=';')
cities_names = [c.lower() for c in cities['Город']]

In [ ]:
cities.set_index('Город', inplace=True)
cities = cities[~cities.index.duplicated(keep='first')]
cities

,Город,Регион,Федеральный округ,lat,lng
0,Абаза,Хакасия,Сибирский,"52,651657","90,088572"
1,Абакан,Хакасия,Сибирский,"53,721152","91,442387"
2,Абдулино,Оренбургская область,Приволжский,"53,677839","53,647263"
3,Абинск,Краснодарский край,Южный,"44,866256","38,151163"
4,Агидель,Башкортостан,Приволжский,"55,899835","53,92204"
...,...,...,...,...,...
1077,Ярославль,Ярославская область,Центральный,"57,626569","39,893787"
1078,Ярцево,Смоленская область,Центральный,"57,535951","40,006121"
1079,Ясногорск,Тульская область,Центральный,"54,479555","37,689689"
1080,Ясный,Оренбургская область,Приволжский,"51,036877","59,874349"


In [ ]:
data

,text,date
0,выясниться мегана марклый свадьба угощать гост...,"30.12.2018, 23:36"
1,mercedes сбить шестеро пешеход невский проспек...,"30.12.2018, 23:05"
2,главный политический тренд год санкциитема сан...,"30.12.2018, 22:58"
3,laquo аквамена raquo вновь возглавить американ...,"30.12.2018, 21:55"
4,берег остров итуруп завалить тонна селёдка огр...,"30.12.2018, 21:49"
...,...,...
22169,жертва новогодний теракт стамбул стать человек...,"01.01.2017, 04:42"
22170,какой страна отмечать новый год январь большин...,"31.12.2016, 17:20"
22171,запад отреагировать решение путин высылать дип...,"31.12.2016, 16:17"
22172,происходить национальный валюта год многий стр...,"31.12.2016, 15:44"


In [ ]:
w = [len(t.split()) for t in data.text]
w1 = sum(w)
w1

5976891

In [ ]:
pc = [[token for token in str(doc).split() if len(token)>2 and token not in ['какой', 'год', 'это', 'также', 'россия'] and token not in cities_names] for doc in data.text]

d = Dictionary(pc)
d.filter_extremes(no_below=15, no_above=0.6)
d.compactify()
bow = [d.doc2bow(text) for text in pc]
lda = LdaModel(bow, id2word=d,
               num_topics=30, 
               random_state=0, 
               chunksize=1740, 
               alpha='auto', eta='auto', 
               iterations=500, passes=20, eval_every=None)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


## Извлекаем таблицу тем

In [ ]:
def term_topic(model):
  topics = [[(term, round(wt, 3)) 
                for term, wt in model.show_topic(n, topn=30)] 
                    for n in range(0, model.num_topics)]
  topics_df = pd.DataFrame([[term for term, wt in topic] 
                                for topic in topics], 
                          columns = ['Term'+str(i) for i in range(1, 31)],
                          index=['Topic '+str(t) for t in range(1, model.num_topics+1)]).T
  return topics_df

def tfidf_corpus(bow):
  tfidf = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
  return tfidf.fit_transform([sum([[t]*q for t, q in bow_doc], []) for bow_doc in bow]).toarray()

def tfidf_sort(df):
  corpus = df.to_numpy().T
  d = Dictionary(corpus)
  bow = [d.doc2bow(text) for text in corpus]
  tfc = tfidf_corpus(bow)
  sorted_tt = np.array([[d[i] for w, i in sorted(((w, i) for i,w in enumerate(doc)), reverse=True) if w > 0] for doc in tfc]).T
  return pd.DataFrame(sorted_tt, 
                      columns = ['Topic'+str(i) for i in range(1, sorted_tt.shape[1]+1)],
                      index=['Term '+str(t) for t in range(1, sorted_tt.shape[0]+1)])

In [ ]:
tt = term_topic(lda)
tt = tfidf_sort(tt)
tt.to_csv('gzetaru_topics.csv',encoding='utf-8-sig')
tt

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30
Term 1,шоу,трагедия,экономический,французский,часть,хакер,цель,уголовный,эрдоган,чёрный,язык,штат,число,хотеть,список,церемония,улица,цена,школа,ход,украинский,ураган,тяжёлый,цвет,фонд,хороший,цик,чиновник,юлия,считать
Term 2,участник,самолёт,транзит,франция,станция,технология,техника,убийство,удар,фсб,сталин,трамп,регион,сделать,санкция,театр,транспортный,ставка,университет,саммит,украинец,тротуар,состояние,фестиваль,федеральный,фильм,церковь,управление,химический,совет
Term 3,участие,рейс,строительство,протест,спутник,социальный,стратегический,теракт,турция,флот,ссср,соединить,крупный,сам,санкционный,сын,скорость,спрос,тело,путин,украина,твиттер,соболезнование,температура,составить,сыграть,собор,руководитель,солсбери,принять
Term 4,сцена,пожарный,северный,парламент,солнечный,смартфон,ракетный,судебный,турецкий,учение,советский,сми,количество,просто,попасть,супруг,руль,снижение,родитель,песок,самопровозглашённый,сегодня,смерть,столица,реформа,сериал,рпц,работать,скрипаль,право
Term 5,рэпер,пожар,предприятие,немецкий,роскосмос,сеть,пентагон,суд,террорист,судный,собчак,обама,житель,поэтому,ограничение,семья,район,прогноз,пациент,переговоры,республика,публикация,скорый,праздник,расход,сезон,президентский,образование,скрипалить,мнение
Term 6,песня,пилот,поток,национальный,рогозин,сервис,оружие,следствие,сирия,судно,русский,конгресс,высокий,почему,мера,отец,полицейский,отмечать,оказаться,обсудить,рада,посмотреть,скончаться,погода,размер,роль,православный,назначить,расследование,возможность
Term 7,певица,пассажир,поставка,макрон,поверхность,приложение,оборона,следственный,сирийский,пролив,память,кндр,больший,очень,лицо,несколько,местный,нефтяной,обнаружить,кремль,пётр,объект,рак,ожидаться,повышение,режиссёр,патриархат,министр,посольство,случай
Term 8,певец,крушение,нафтогаз,италия,планета,пользователь,нато,следователь,организация,порт,памятник,китай,тысяча,нужно,иностранный,имя,машина,нефть,найти,днр,порошенко,недвижимость,помощь,одежда,пенсия,прокат,навальный,министерство,официальный,эксперт
Term 9,музыкант,катастрофа,млрд,германия,орбита,мобильный,минобороны,признать,оон,подводный,отечественный,ким,тыс,наш,запрет,известный,дтп,курс,мужчина,два,полуостров,метр,минздрав,ночью,пенсионный,премьера,московский,заместитель,отравление,ситуация
Term 10,музыкальный,воздушный,мвф,армения,миссия,клиент,комплекс,преступление,исламский,пограничник,николай,дональд,развитие,знать,ввести,жена,дорожный,инфляция,исследователь,встреча,лнр,квартира,медицинский,ночь,налоговый,лента,кандидат,должность,мэй,закон


## Разметим города по упоминанию в документах

In [ ]:
city_to_doc = {city : [i for i in range(data.shape[0]) if city in data.text.iloc[i].split()]
               for city in cities_names}

city_to_doc = {k : v for k,v in city_to_doc.items() if len(v) > 0}

In [ ]:
topics = [0,1,5,7,16,17,22,24,25,27]

In [ ]:
doc_to_topic = [[top for top, w in lda.get_document_topics(bow[i]) if top in topics and w>0.05] for i in range(data.shape[0])]

In [ ]:
from collections import Counter

city_to_topic = {k:dict(Counter(sum([doc_to_topic[d] for d in v], []))) for k,v in city_to_doc.items()}

city_to_topic = {k : {k1:v1 for k1, v1 in v.items() if v1>1}
                 for k,v in city_to_topic.items()}
city_to_topic = {k : v for k,v in city_to_topic.items() if len(v)>0}

In [ ]:
df0 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df1 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df5 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df7 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df16 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df17 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df22 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df24 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df25 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
df27 = pd.DataFrame(columns=['City','Lat', 'Lon', 'N'])
dfs = [df0,df1,df5,df7,df16,df17,df22,df24,df25,df27]

In [ ]:
for t,df in zip(topics,dfs):
  for k,v in city_to_topic.items():
    if t in v.keys():
      df.loc[df.shape[0]] = {'City' : k.capitalize(),
                              'Lat': str(cities.loc[k.capitalize()].lat).replace(',', '.'),
                              'Lon': (cities.loc[k.capitalize()].lng).replace(',', '.'),
                              'N': v[t]}

In [ ]:
topic_names = ['Музыка', 'Катастрофы', 'Технологии', 'Уголовная_хроника', 'ДТП', 'Экономика', 'Здравоохранение_смерть', 'Соц_поддержка', 'Кино', 'Госуправление']

In [ ]:
#import shutil
shutil.rmtree('/content/gdrive/MyDrive/DH_topic_layers_txt')
mkdir('/content/gdrive/MyDrive/DH_topic_layers_txt')

In [ ]:
dr = '/content/gdrive/MyDrive/DH_topic_layers_txt'
for tname, df in zip(topic_names, dfs):
  df.to_csv(join(dr, tname +'.txt'),index=False, encoding='utf-8', sep=';')